# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import train_xgb # The module for loading external data and training an XGB model
import os
import pandas as pd
import numpy as np
import json
import ast
import pickle

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Dataset, Experiment, Model, Environment, ScriptRunConfig
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails

from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, loguniform, choice

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

quick-starts-ws-154360
aml-quickstarts-154360
southcentralus
3d1a56d2-7c81-4118-9790-f85d1acf0c77


In [3]:
# Create compute cluster
# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           min_nodes=1,
                                                           max_nodes=5)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress.....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded...............................................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
# # Try to load the dataset from the workspace. Otherwise, load if from Kaggle
# found = False
# ds_key = 'Ames-housing-dataset'
# ds_desc = 'Ames Housing training data.'

# if ds_key in ws.datasets.keys():
#     found = True
#     dataset = ws.datasets[ds_key]
#     print(f'Found registered {ds_key}, use it.')
    
# if not found:
#     train, test = ames.load_data_clean()
#     print(f"train.shape = {train.shape}, test.shape = {test.shape}")
#     # Register the train dataset
#     blob = ws.get_default_datastore()
#     dataset = TabularDatasetFactory.register_pandas_dataframe(train, blob, name=ds_key, description=ds_desc)

In [5]:
# Test the train script
# ! python train_xgb.py

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
# Choose a name for an experiment
experiment_name = 'Ames-housing-hdr'

experiment=Experiment(ws, experiment_name)

In [7]:
%%writefile conda_env.yml

dependencies:
- python=3.6.2
- pip:
  - azureml-defaults==1.32.0
- scikit-learn
- xgboost

Writing conda_env.yml


In [8]:
# Define an Azure ML environment
# Dependencies are the same as for AutoML experiment
env = Environment.from_conda_specification(name='env', file_path='conda_env.yml')

# Configure the training job
src = ScriptRunConfig(source_directory=".",
                     script='train_xgb.py',
                     #arguments=['--learning_rate', 0.01, '--gamma', 5, '--max_depth', 5], # Just for testing
                     compute_target=cpu_cluster,
                     environment=env)

In [9]:
# Test the script run config
# run = experiment.submit(src)

In [10]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--learning_rate': loguniform(-4.6, -1.6), # results in [0.01, 0.2]
        '--gamma': uniform(0, 9), 
        '--max_depth': choice(3, 5, 7)
    }
)

#TODO: Create your estimator and hyperdrive config
# src - see above

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='r2_score',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=20,
                                    max_concurrent_runs=4,
                                    max_duration_minutes=30)

In [11]:
#TODO: Submit your experiment
hdr = experiment.submit(config=hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
# Show run details with the widget.
RunDetails(hdr).show()
hdr.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_51828b91-12c3-437c-b8dc-c6f94e585c2a
Web View: https://ml.azure.com/runs/HD_51828b91-12c3-437c-b8dc-c6f94e585c2a?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-154360/workspaces/quick-starts-ws-154360&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-08-12T13:11:00.271627][API][INFO]Experiment created<END>\n""<START>[2021-08-12T13:11:00.819600][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-08-12T13:11:01.017029][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_51828b91-12c3-437c-b8dc-c6f94e585c2a
Web View: https://ml.azure.com/runs/HD_51828b91-12c3-437c-b8dc-c6f94e585c2a?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-154360/workspaces/quick-starts-ws-154360&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_51828b91-12c3-437c-b8dc-c6f94e585c2a',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-12T13:10:59.925072Z',
 'endTimeUtc': '2021-08-12T13:27:12.830294Z',
 'properties': {'primary_metric_config': '{"name": "r2_score", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'e662eaf7-dad9-4aa5-a45a-8a84b8c3a70e',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.32.0',
  'score': '0.8883161946170155',
  'best_child_run_id': 'HD_51828b91-12c3-437c-b8dc-c6f94e585c2a_13',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg154360.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_51828b91-12c3-437c-b8dc-c6f94e585c2a/azureml-logs/hyperdrive.txt?sv=2019-07-07&sr=b&

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [13]:
# Get your best run and save the model from that run.
best_run = hdr.get_best_run_by_primary_metric()
print(best_run)

Run(Experiment: Ames-housing-hdr,
Id: HD_51828b91-12c3-437c-b8dc-c6f94e585c2a_13,
Type: azureml.scriptrun,
Status: Completed)


In [14]:
best_run_metrics = best_run.get_metrics()
best_run_metrics

{'Learning rate': 0.13121033568793716,
 'Gamma': 1.6125750464873576,
 'Maximum depth': 5.0,
 'r2_score': 0.8883161946170155}

In [18]:
details = best_run.get_details()

# Save metrics and details for ex-post examination
os.makedirs('./outputs', exist_ok=True)
with open('outputs/best_hdr_metrics.json', 'w') as file:
    json.dump(best_run_metrics, file)
with open('outputs/best_hdr_details.txt', 'w') as file:
    file.write(str(details))

In [19]:
best_run.get_file_names()[-1]

'outputs/model.joblib'

In [20]:
#TODO: Save the best model
best_run.download_file(best_run.get_file_names()[-1], output_file_path='./outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [21]:
# Register the best model
model = Model.register(ws, model_path='outputs/model.joblib', model_name='Ames-Housing-XGB-Model', tags=best_run_metrics)
print(model.name, model.id, model.version, sep='\t')

Registering model Ames-Housing-XGB-Model
Ames-Housing-XGB-Model	Ames-Housing-XGB-Model:1	1


In [22]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                              memory_gb=1,
                                              tags={"data" : "Kaggle", "method" : "XGB"},
                                              description="Predict Ames Housing Prices",
                                              auth_enabled=True,
                                              enable_app_insights=True)

In [23]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment("project-env")
cd = CondaDependencies('conda_env.yml')
env.python.conda_dependencies = cd
# Register environment to re-use later
env.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210615.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "project-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "dependencies": [
                "python=3.6.2",
                {
      

In [24]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'Ames-Housing-XGB-Model')

myenv = Environment.get(workspace=ws, name="project-env", version="1")
inference_config = InferenceConfig(entry_script="entry_script.py", environment=myenv)

service_name = 'ames-housing-xgb-' + str(uuid.uuid4())[:4]
service = Model.deploy(workspace=ws,
                      name=service_name,
                      models=[model],
                      inference_config=inference_config,
                      deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-12 13:38:23+00:00 Creating Container Registry if not exists.
2021-08-12 13:38:23+00:00 Registering the environment.
2021-08-12 13:38:25+00:00 Use the existing image.
2021-08-12 13:38:25+00:00 Generating deployment configuration.
2021-08-12 13:38:26+00:00 Submitting deployment to compute.
2021-08-12 13:38:31+00:00 Checking the status of deployment ames-housing-xgb-61d4..
2021-08-12 13:42:16+00:00 Checking the status of inference endpoint ames-housing-xgb-61d4.
Succeeded
ACI service creation operation finished, operation "Succeeded"
CPU times: user 2.72 s, sys: 187 ms, total: 2.91 s
Wall time: 4min


In [25]:
print(service.get_logs())

2021-08-12T13:42:07,006913500+00:00 - nginx/run 
2021-08-12T13:42:07,015714700+00:00 - rsyslog/run 
2021-08-12T13:42:07,021009600+00:00 - iot-server/run 
2021-08-12T13:42:07,049242800+00:00 - gunicorn/run 
File not found: /var/azureml-app/.
Starting HTTP server
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-08-12T13:42:07,332114300+00:00 - iot-server/finish 1 0
2021-08-12T13:42:07,334358800+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (66)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 91
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-08-12 13:42:08,583 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-08-12 13:42:08,584 | root | INFO | Starting up request id generator
2021-08-12 13:42:08,584 | root | INFO | Starting up app in

TODO: In the cell below, send a request to the web service you deployed to test it.

In [26]:
service.scoring_uri

'http://ebd21c0e-165a-4780-ac38-e452588e8526.southcentralus.azurecontainer.io/score'

In [27]:
_, test = train_xgb.load_data_clean()
test = train_xgb.label_encode(test)

In [28]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In [29]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
data = {
        "data":
    [
        test.iloc[0].to_dict(),
        test.iloc[1].to_dict(),
    ],
}

body = str.encode(json.dumps(data))

url = 'http://ebd21c0e-165a-4780-ac38-e452588e8526.southcentralus.azurecontainer.io/score'
api_key = 'WyqjmK0i1DGGkWzOnDqn1gy18ZDYM3dZ' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

The request failed with status code: 502
Content-Length: 5600
Content-Type: text/html; charset=utf-8
Date: Thu, 12 Aug 2021 13:49:05 GMT
Server: nginx/1.14.0 (Ubuntu)
X-Ms-Request-Id: f628f63d-66b1-4189-a3dd-c380326fe802
X-Ms-Run-Function-Failed: True
Connection: close




JSONDecodeError: Expecting ',' delimiter: line 1 column 4 (char 3)

In [30]:
req.data

b'{"data": [{"MSSubClass": 0.0, "MSZoning": 2.0, "LotFrontage": 80.0, "LotArea": 11622.0, "Street": 1.0, "Alley": 2.0, "LotShape": 1.0, "LandContour": 3.0, "Utilities": 3.0, "LotConfig": 4.0, "LandSlope": 3.0, "Neighborhood": 12.0, "Condition1": 1.0, "Condition2": 2.0, "BldgType": 0.0, "HouseStyle": 2.0, "OverallQual": 6.0, "OverallCond": 7.0, "YearBuilt": 1961.0, "YearRemodAdd": 1961.0, "RoofStyle": 1.0, "RoofMatl": 0.0, "Exterior1st": 10.0, "Exterior2nd": 13.0, "MasVnrType": 2.0, "MasVnrArea": 0.0, "ExterQual": 3.0, "ExterCond": 3.0, "Foundation": 1.0, "BsmtQual": 3.0, "BsmtCond": 3.0, "BsmtExposure": 1.0, "BsmtFinType1": 3.0, "BsmtFinSF1": 468.0, "BsmtFinType2": 2.0, "BsmtFinSF2": 144.0, "BsmtUnfSF": 270.0, "TotalBsmtSF": 882.0, "Heating": 0.0, "HeatingQC": 3.0, "CentralAir": 2.0, "Electrical": 5.0, "1stFlrSF": 896.0, "2ndFlrSF": 0.0, "LowQualFinSF": 0.0, "GrLivArea": 896.0, "BsmtFullBath": 0.0, "BsmtHalfBath": 0.0, "FullBath": 1.0, "HalfBath": 0.0, "BedroomAbvGr": 2.0, "KitchenAbvG

In [40]:
import joblib
xgb = joblib.load('outputs/model.joblib')

TODO: In the cell below, print the logs of the web service and delete the service

In [38]:
service.delete()
# Delete() is used to deprovision and delete the AmlCompute target. 
cpu_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

